In [1]:
import pandas as pd
import data_util
from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from state_util import StateUtil
from tec_an import TecAn
from data_agent import DataAgent, TacProcess
import numpy as np
from data_util import *
import tensorflow as tf
from sklearn_model_hyper import *
import pandas as pd
from stock_agent import *


In [2]:
def get_y_data(ohlc, shift = -1):
    combined_data = ohlc
    #combined_data['return'] = np.log(combined_data / combined_data.shift(1))
    returns = (ohlc / ohlc.shift(shift))
    combined_data['return'] = returns
    combined_data['direction'] = np.where(combined_data['return'] < 1, 1, 0)
    #print(combined_data)
    #combined_data.dropna(inplace=True)
    return combined_data['direction']

def split(x, y, split, shuffle=False):
    trainX, valX, trainY, valY = train_test_split(np.array(x), np.array(y), test_size=split, shuffle=shuffle)
    print("Completed: {} {} {} {}".format(trainX.shape, trainY.shape, valX.shape, valY.shape))
    return trainX, trainY, valX, valY


def get_full_database(resample, raw_dir, base_dir = "data/"):

    full_data = base_dir + raw_dir + "/"
    data_gen = DataGenerator(random = False, base_dir = full_data)
    data_gen.rewind()
    data_count = (data_gen.steps - 100)
    #data_count = 20000

    final_x = []

    closed_prices = []

    on_new_data = lambda x: final_x.append(x)
    on_closed_price = lambda price: closed_prices.append(price)
    
    agent = DataAgent(
        resample = resample,
        on_new_data = on_new_data,
        on_closed_price = on_closed_price
    )

    print("Processing {}".format(raw_dir))

    for i in tqdm(range(data_count)):
        agent.on_new_raw_data(data_gen.next())


    closes = pd.DataFrame(closed_prices, columns = ['Close'])

    final_y = get_y_data(closes, -2)
    
    #print(agent.ohlc)
    
    return final_x, final_y.to_numpy(), closed_prices

In [3]:
data_gen = DataGenerator(random=False, base_dir = "data/4jul21/")

#data_gen = DataGenerator(random=False, base_dir = "data/backtest/")

#data_gen = DataGenerator(random=False, base_dir = "data/btcusd/")

data_gen.rewind()


In [4]:
from joblib import dump, load

models = [
    load('model/StackingClassifier'), 
    load('model/7142857142857143'),
    load('model/RandomForestClassifier'),
    load('model/RandomForestClassifier_accuracy_score'),
    load('model/RandomForestClassifier_f1_score'),
    load('model/QuadraticDiscriminantAnalysis'),
    load('model/RandomForestClassifier2')
]

print(f"{len(models)}")

7


In [5]:

x, y, closed_prices = get_full_database('2Min', "4jul21", base_dir = "data/")

len(x)

Resample 2Min - TecAn ( windows 20, windows_limit 100 )
Processing 4jul21


<ipython-input-2-3dd1fc32f772>:40: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(data_count)):


  0%|          | 0/301794 [00:00<?, ?it/s]

301

In [10]:
back = BackTest(value = 100, verbose = True)
for idx in range(len(x)):
    yy = y[idx]
    price = closed_prices[idx]
    #print(f'{idx} {yy} {price}')
    back.on_state(0, price)
    if(yy == 1):
        back.request_buy(price)
    else:
        back.request_sell(price)


1970-01-01 00:00:00 Buy (35380.96): ask: 35380.96
1970-01-01 00:00:00 Sell (35413.55) 32.59000000000378 total: 100.09211168945106
1970-01-01 00:00:00 Buy (35250.24): ask: 35250.24
1970-01-01 00:00:00 Sell (35325.67) 75.43000000000029 total: 100.30609604813912
1970-01-01 00:00:00 Buy (35313.7): ask: 35313.7
1970-01-01 00:00:00 Sell (35425.94) 112.24000000000524 total: 100.62393303491766
1970-01-01 00:00:00 Buy (35383.52): ask: 35383.52
1970-01-01 00:00:00 Sell (35433.55) 50.03000000000611 total: 100.76532654240364
1970-01-01 00:00:00 Buy (35281.97): ask: 35281.97
1970-01-01 00:00:00 Sell (35338.36) 56.38999999999942 total: 100.92515321874853
1970-01-01 00:00:00 Buy (35331.87): ask: 35331.87
1970-01-01 00:00:00 Sell (35333.93) 2.0599999999976717 total: 100.93098364889559
1970-01-01 00:00:00 Buy (34929.17): ask: 34929.17
1970-01-01 00:00:00 Sell (35019.56) 90.38999999999942 total: 101.18976449023822
1970-01-01 00:00:00 Buy (34949.27): ask: 34949.27
1970-01-01 00:00:00 Sell (35006.43) 57.1

In [8]:
back.report()

100
108.22014764126742
108.22014764126742


In [7]:
data_count = data_gen.steps
#data_count = 200000 
data_gen.rewind()

raw_data = []

for i in tqdm(range(data_count)):
    raw_data.append(data_gen.next())

<ipython-input-7-5fbf08f74a14>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(data_count)):


  0%|          | 0/301894 [00:00<?, ?it/s]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-5fbf08f74a14>", line 8, in <module>
    raw_data.append(data_gen.next())
  File "/Users/samirantonio/Desktop/YaStockRnn/data_generator.py", line 52, in next
    state = self.get_current_state()
  File "/Users/samirantonio/Desktop/YaStockRnn/data_generator.py", line 46, in get_current_state
    return self.get_from_index(self.index)
  File "/Users/samirantonio/Desktop/YaStockRnn/data_generator.py", line 34, in get_from_index
    return self.read_json_from_file(file_path)
  File "/Users/samirantonio/Desktop/YaStockRnn/data_generator.py", line 27, in read_json_from_file
    return json.loads(raw)
  File "/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  Fil

TypeError: object of type 'NoneType' has no len()

In [ ]:
def process_model(model, resample):
    back = BackTest(value = 100)

    request_sell = lambda price: back.request_sell(price)
    request_buy = lambda price: back.request_buy(price)
    on_state = lambda timestamp, price: back.on_state(timestamp, price)

    stock = StockAgent(model = model,
                    request_sell = request_sell,
                    request_buy = request_buy,
                    on_state = on_state
                      )


    data_gen.rewind()
    back.reset()

    on_new_data = lambda x: print(x)
    on_new_data = lambda x: stock.on_x(x)

    on_state = lambda timestamp, price, bid, ask: print("{} {} {} {}".format(timestamp, price, bid[-1], ask[-1]))
    on_state = lambda timestamp, price, bid, ask: stock.on_new_state(timestamp, price, bid, ask)


    agent = DataAgent(
        resample = resample,
        on_state = on_state,
        on_new_data = on_new_data
    )

    on_new_data_count = 0

    for raw in tqdm(raw_data):
        agent.on_new_raw_data(raw)

    if (back.bought):
        stock.sell()

    (agent.ohlc).plot(figsize=(20, 12))
        
    return back

In [ ]:
class ModelTest():
    
    def __init__(self,
                 resample = '2Min'
                 ):
        self.index = 0
        
    def predict(self, x):
        yy = y[self.index]
        self.index += 1
        return [yy]

In [ ]:
model = ModelTest()
backProc = process_model(model, '2Min')

In [ ]:
backProc.report()

In [ ]:
num += 1

In [ ]:
model = ModelTest()
backProc = process_model(model, '2Min')